In [7]:
from  TDClient import FData
import time
import pandas as pd
import datetime
import pytz
indian_tz = pytz.timezone('Asia/Kolkata')
import warnings
warnings.filterwarnings("ignore")

username = 'password = '
nurl = 'http://3.6.166.159/td'                 
url = "http://127.0.0.1:5000"
#nurl='http://192.168.2.173:6969'

td_obj = FData(key='trial', base_url=nurl)
td_obj.login()
td_obj.connect_to_server(username=username, password=password, realtime_port=8082, url='push.truedata.in')

<Response [200]>

In [16]:
import requests

class OrderWebhook:
    def __init__(self):
        self.uri = "http://3.6.166.159/fpt/"

    def placeorder(self, buyorsell, day, month, year, side, strike_price, symbol):
        a = requests.post(self.uri + "makeorder",
                          json={"buyorsell": buyorsell, "day": day, "month": month, "year": year, "side": side,
                                "strike_price": strike_price, "symbol": symbol})
        return a.json()

obj = OrderWebhook()

In [8]:
from sqlalchemy.sql.expression import true

sym = 'BANKNIFTY'
year,month,day,length = 2023,7,20,40

start_time = datetime.time(14,50)
end_time = datetime.time(15, 27)
current_time = datetime.datetime.now(indian_tz).time()

trade_entry = []
trade_exit = []
live_pnl = []
profit = []

trade = False

In [9]:
td_obj.add_option_symbols(sym, year, month, day,length)
time.sleep(5)

In [19]:
while (current_time>start_time)&(current_time<end_time):
    
    try:
    
        df = pd.DataFrame(td_obj.get_oc_data()[sym + str(year) + str(month)+ str(day)])
        df = df[['ltp', 'ask','bid', 'ltt', 'strike', 'type']]
        df.index = df['strike']
        df= df.drop(columns =['strike'])
        df.ltt = pd.to_datetime(df.ltt)
        df = df[(df['ask']!=0)&(df['bid']!=0)]

        df_ce = df[df['type'] == 'CE']
        df_pe = df[df['type'] == 'PE']

        if (current_time<datetime.time(14,59,30)):

            ce_strike = abs(df_ce['ask'] - 100).sort_values().index[0]
            ce_premium = df_ce[df_ce.index == ce_strike]['ask'][0]

            ce_limit = ce_premium * 0.9
            ce_sl = ce_premium*1.25

            pe_strike = abs(df_pe['ask'] - 100).sort_values().index[0]
            pe_premium = df_pe[df_pe.index == pe_strike]['ask'][0]

            pe_limit = pe_premium * 0.9
            pe_sl = pe_premium*1.25

            hedge_ce_strike = abs(df_ce['ask'] - 5).sort_values().index[0]
            hedge_ce_premium = df_ce[df_ce.index == hedge_ce_strike]['ask'][0]
            
            hedge_pe_strike = abs(df_pe['ask'] - 5).sort_values().index[0]
            hedge_pe_premium = df_pe[df_pe.index == hedge_pe_strike]['ask'][0]
            

        if (current_time>datetime.time(15,0)):                          

            live_ce_prem = df_ce[df_ce.index == ce_strike]['ask'][0]
            live_pe_prem = df_pe[df_pe.index == pe_strike]['ask'][0]

            print('CE',live_ce_prem,'PE',live_pe_prem)

            if (live_ce_prem < ce_limit) & (trade == False):

                trade = True
                
                #Hedges entry
                
                obj.placeorder(buyorsell="B", day=day, month=month, year=year, 
                                side="CE", strike_price=hedge_ce_strike, symbol=sym)
                
                obj.placeorder(buyorsell="B", day=day, month=month, year=year, 
                                side="PE", strike_price=hedge_pe_strike, symbol=sym)
                
                time.sleep(5)
                
                #Main Position Entry
                    
                obj.placeorder(buyorsell="S", day=day, month=month, year=year, 
                                side="CE", strike_price=ce_strike, symbol=sym)

                trade_entry.append(['CE Entry', live_ce_prem, ce_strike, ce_limit, ce_sl])

            if (live_pe_prem < pe_limit) & (trade == False):

                trade = True
                
                #Hedges entry
                
                obj.placeorder(buyorsell="B", day=day, month=month, year=year, 
                                side="CE", strike_price=hedge_ce_strike, symbol=sym)
                
                obj.placeorder(buyorsell="B", day=day, month=month, year=year, 
                                side="PE", strike_price=hedge_pe_strike, symbol=sym)
                
                time.sleep(5)
                
                #Main Position Entry
                    
                obj.placeorder(buyorsell="S", day=day, month=month, year=year, 
                                side="PE", strike_price=pe_strike, symbol=sym)

                trade_entry.append(['PE Entry', live_pe_prem, pe_strike, pe_limit, pe_sl])


        if trade == True:

            if trade_entry[-1][0] == 'CE Entry':

                live_ce_prem = df_ce[df_ce.index == ce_strike]['ask'][0]

                print(current_time, 'Live CE Profit:', (trade_entry[-1][1] - live_ce_prem)*25)

                if live_ce_prem > ce_sl:

                    trade = False
                    
                    #Main Position Exit

                    obj.placeorder(buyorsell="B", day=day, month=month, year=year, 
                                    side="CE", strike_price=ce_strike, symbol=sym)
                    
                    time.sleep(5)
                    
                    #Hedges Exit

                    obj.placeorder(buyorsell="S", day=day, month=month, year=year, 
                                    side="CE", strike_price=hedge_ce_strike, symbol=sym)

                    obj.placeorder(buyorsell="S", day=day, month=month, year=year, 
                                    side="PE", strike_price=hedge_pe_strike, symbol=sym)

                    
                    trade_exit.append(['CE Entry', live_ce_prem])

                    profit.append((trade_entry[-1][1] - trade_exit[-1][1])*25)

                    break

                elif current_time>datetime.time(15,26):
                    
                    
                    #Main Position Exit

                    obj.placeorder(buyorsell="B", day=day, month=month, year=year, 
                                    side="CE", strike_price=ce_strike, symbol=sym)
                    
                    time.sleep(5)
                    
                    #Hedges Exit

                    obj.placeorder(buyorsell="S", day=day, month=month, year=year, 
                                    side="CE", strike_price=hedge_ce_strike, symbol=sym)

                    obj.placeorder(buyorsell="S", day=day, month=month, year=year, 
                                    side="PE", strike_price=hedge_pe_strike, symbol=sym)


                    trade_exit.append(['CE Entry', live_ce_prem])

                    profit.append((trade_entry[-1][1] - trade_exit[-1][1])*25)

                    break

        if trade == True:

            if trade_entry[-1][0] == 'PE Entry':

                live_pe_prem = df_pe[df_pe.index == pe_strike]['ask'][0]

                print(current_time, 'Live PE Profit:', (trade_entry[-1][1] - live_pe_prem)*25)

                if live_pe_prem > pe_sl:

                    trade = False

                    trade_exit.append(['PE Entry', live_pe_prem])

                    profit.append((trade_entry[-1][1] - trade_exit[-1][1])*25)
                    
                    
                    #Main Position Exit

                    obj.placeorder(buyorsell="B", day=day, month=month, year=year, 
                                    side="PE", strike_price=pe_strike, symbol=sym)
                    
                    time.sleep(5)
                    
                    #Hedges Exit

                    obj.placeorder(buyorsell="S", day=day, month=month, year=year, 
                                    side="CE", strike_price=hedge_ce_strike, symbol=sym)

                    obj.placeorder(buyorsell="S", day=day, month=month, year=year, 
                                    side="PE", strike_price=hedge_pe_strike, symbol=sym)


                    break

                elif current_time>datetime.time(15,24,30):
                    
                    #Main Position Exit

                    obj.placeorder(buyorsell="B", day=day, month=month, year=year, 
                                    side="PE", strike_price=pe_strike, symbol=sym)
                    
                    time.sleep(5)
                    
                    #Hedges Exit

                    obj.placeorder(buyorsell="S", day=day, month=month, year=year, 
                                    side="CE", strike_price=hedge_ce_strike, symbol=sym)

                    obj.placeorder(buyorsell="S", day=day, month=month, year=year, 
                                    side="PE", strike_price=hedge_pe_strike, symbol=sym)

                    trade_exit.append(['PE Entry', live_pe_prem])

                    profit.append((trade_entry[-1][1] - trade_exit[-1][1])*25)

                    break

        time.sleep(10)
        current_time = datetime.datetime.now(indian_tz).time()
        
    except Exception as e:
        
        td_obj = FData(key='trial', base_url=nurl)
        td_obj.login()
        td_obj.connect_to_server(username=username, password=password, realtime_port=8082, url='push.truedata.in')
        
        td_obj.add_option_symbols(sym, year, month, day,length)
        time.sleep(5)
        
        print("Restart")
        pass

CE 113.3 PE 86.85
CE 115.8 PE 85.55
CE 115.7 PE 84.4
CE 120.05 PE 77.95
15:00:35.584380 Live PE Profit: 0.0
CE 118.65 PE 80.45
15:00:51.748911 Live PE Profit: -62.5
CE 117.6 PE 82.4
15:01:01.880704 Live PE Profit: -111.25000000000007
CE 108.85 PE 87.0
15:01:12.071892 Live PE Profit: -226.24999999999994
CE 111.2 PE 86.05
15:01:22.240320 Live PE Profit: -202.49999999999986
CE 114.35 PE 85.5
15:01:32.397199 Live PE Profit: -188.74999999999994
CE 117.0 PE 82.1
15:01:42.541497 Live PE Profit: -103.74999999999979
CE 115.3 PE 84.75
15:01:53.607104 Live PE Profit: -169.99999999999994
CE 117.0 PE 83.25
15:02:03.741775 Live PE Profit: -132.49999999999994
Restart
CE 110.95 PE 85.95
15:02:13.887634 Live PE Profit: -200.0
CE 114.4 PE 84.9
15:03:06.285849 Live PE Profit: -173.75000000000006
CE 114.5 PE 84.15
15:03:16.405870 Live PE Profit: -155.00000000000006
CE 115.3 PE 83.1
15:03:26.554230 Live PE Profit: -128.74999999999977
CE 115.25 PE 83.35
15:03:36.704837 Live PE Profit: -134.99999999999977
CE

In [10]:
while (current_time>start_time)&(current_time<end_time):
    
    try:
    
        df = pd.DataFrame(td_obj.get_oc_data()[sym + str(year) + str(month)+ str(day)])
        df = df[['ltp', 'ask','bid', 'ltt', 'strike', 'type']]
        df.index = df['strike']
        df= df.drop(columns =['strike'])
        df.ltt = pd.to_datetime(df.ltt)
        df = df[(df['ask']!=0)&(df['bid']!=0)]

        df_ce = df[df['type'] == 'CE']
        df_pe = df[df['type'] == 'PE']

        if (current_time<datetime.time(14,59,30)):

            ce_strike = abs(df_ce['ask'] - 100).sort_values().index[0]
            ce_premium = df_ce[df_ce.index == ce_strike]['ask'][0]

            ce_limit = ce_premium * 0.9
            ce_sl = ce_premium*1.25

            pe_strike = abs(df_pe['ask'] - 100).sort_values().index[0]
            pe_premium = df_pe[df_pe.index == pe_strike]['ask'][0]

            pe_limit = pe_premium * 0.9
            pe_sl = pe_premium*1.25

            hedge_ce_strike = abs(df_ce['ask'] - 5).sort_values().index[0]
            hedge_ce_premium = df_ce[df_ce.index == hedge_ce_strike]['ask'][0]
            
            hedge_pe_strike = abs(df_pe['ask'] - 5).sort_values().index[0]
            hedge_pe_premium = df_pe[df_pe.index == hedge_pe_strike]['ask'][0]
            

        if (current_time>datetime.time(15,0)):                          

            live_ce_prem = df_ce[df_ce.index == ce_strike]['ask'][0]
            live_pe_prem = df_pe[df_pe.index == pe_strike]['ask'][0]

            print('CE',live_ce_prem,'PE',live_pe_prem)

            if (live_ce_prem < ce_limit) & (trade == False):

                trade = True
                


                trade_entry.append(['CE Entry', live_ce_prem, ce_strike, ce_limit, ce_sl])

            if (live_pe_prem < pe_limit) & (trade == False):

                trade = True

                trade_entry.append(['PE Entry', live_pe_prem, pe_strike, pe_limit, pe_sl])


        if trade == True:

            if trade_entry[-1][0] == 'CE Entry':

                live_ce_prem = df_ce[df_ce.index == ce_strike]['ask'][0]

                print(current_time, 'Live CE Profit:', (trade_entry[-1][1] - live_ce_prem)*25)

                if live_ce_prem > ce_sl:

                    trade = False
                    


                    
                    trade_exit.append(['CE Entry', live_ce_prem])

                    profit.append((trade_entry[-1][1] - trade_exit[-1][1])*25)

                    break

                elif current_time>datetime.time(15,26):
                    
                    



                    trade_exit.append(['CE Entry', live_ce_prem])

                    profit.append((trade_entry[-1][1] - trade_exit[-1][1])*25)

                    break

        if trade == True:

            if trade_entry[-1][0] == 'PE Entry':

                live_pe_prem = df_pe[df_pe.index == pe_strike]['ask'][0]

                print(current_time, 'Live PE Profit:', (trade_entry[-1][1] - live_pe_prem)*25)

                if live_pe_prem > pe_sl:

                    trade = False

                    trade_exit.append(['PE Entry', live_pe_prem])

                    profit.append((trade_entry[-1][1] - trade_exit[-1][1])*25)
                    
                    



                    break

                elif current_time>datetime.time(15,24,30):


                    trade_exit.append(['PE Entry', live_pe_prem])

                    profit.append((trade_entry[-1][1] - trade_exit[-1][1])*25)

                    break

        time.sleep(10)
        current_time = datetime.datetime.now(indian_tz).time()
        
    except Exception as e:
        
        td_obj = FData(key='trial', base_url=nurl)
        td_obj.login()
        td_obj.connect_to_server(username=username, password=password, realtime_port=8082, url='push.truedata.in')
        
        td_obj.add_option_symbols(sym, year, month, day,length)
        time.sleep(5)
        
        print("Restart")
        pass

CE 93.9 PE 91.0
Restart
CE 96.9 PE 88.1
CE 100.0 PE 86.2
15:01:09.682520 Live PE Profit: 0.0
CE 108.1 PE 79.45
15:01:20.067813 Live PE Profit: 168.75
CE 102.8 PE 81.4
15:01:30.773695 Live PE Profit: 119.99999999999993
CE 107.05 PE 78.4
15:01:41.051957 Live PE Profit: 194.99999999999994
CE 102.9 PE 78.55
15:01:51.432653 Live PE Profit: 191.25000000000014
CE 104.1 PE 79.2
15:02:01.587629 Live PE Profit: 175.0
CE 104.6 PE 77.9
15:02:12.018706 Live PE Profit: 207.49999999999994
CE 107.3 PE 77.9
15:02:22.590444 Live PE Profit: 207.49999999999994
CE 104.55 PE 78.45
15:02:32.761671 Live PE Profit: 193.75
CE 98.95 PE 82.7
15:02:43.764550 Live PE Profit: 87.5
CE 93.05 PE 86.0
15:02:54.460795 Live PE Profit: 5.000000000000071
CE 97.3 PE 83.5
15:03:05.477418 Live PE Profit: 67.50000000000007
CE 96.2 PE 84.7
15:03:15.961606 Live PE Profit: 37.5
CE 95.25 PE 83.5
15:03:26.449321 Live PE Profit: 67.50000000000007
CE 102.2 PE 79.4
15:03:36.814784 Live PE Profit: 169.99999999999994
CE 100.05 PE 81.2
15

In [11]:
trade_entry

[['PE Entry', 86.2, '45500', 87.93, 122.125]]

In [12]:
profit

[200.0]